In [1]:
# import libraries 

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup
import requests
import datetime
import re

In [2]:
# Define your Google Sheets credentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('path/to/credentials.json', scope)
client = gspread.authorize(creds)

In [3]:
sheet = client.open('Amazon Data Analytics').sheet1
skus = sheet.col_values(1)[1:]
urls = sheet.col_values(2)[1:]

In [4]:
# Comment after running the first time

column_names = ['DateTime', 'JewelleryRanking', 'WomensBrooches&PinsRanking']
for sku in skus:
    worksheet = client.open('Amazon Data Analytics').add_worksheet(title=sku, rows="500", cols="20")
    worksheet.append_row(column_names)
    pass

In [5]:
# Code to delete all worksheets except first, Uncomment only when required.

#sheet = client.open("Amazon Data Analytics")
#worksheets = sheet.worksheets()
#main_sheet = worksheets[0]
#for worksheet in worksheets[1:]:
#    sheet.del_worksheet(worksheet)

In [6]:
def pull_rank(sku, url):

    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    response = requests.get(url, headers=headers)

    soup1 = BeautifulSoup(response.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    text='Best Sellers Rank:'

    gfg = soup2.find(lambda tag: tag.name == "span" and text in tag.text).get_text()

    gfg = gfg.replace(',','')

    first_ranking = re.search(r"(?<=#).*?(\d+)", gfg).group(0)
        
    gfg = gfg.replace(first_ranking,'')
    second_ranking = re.search(r"(?<=#).*?(\d+)", gfg).group(0)

    first_ranking = int(first_ranking)
    second_ranking = int(second_ranking)

    time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    data = [time, first_ranking, second_ranking]

    sheet = client.open('Amazon Data Analytics').worksheet(sku)
    sheet.append_row(data)

In [7]:
for sku, url in zip(skus, urls):
    pull_rank(sku, url)